In [2]:
from glob import glob
from PIL import Image
import pandas as pd
from bs4 import BeautifulSoup
import requests, os, imagehash
from src.downloader import download
from multiprocessing.dummy import Pool

def get_html(url):
    """Get raw HTML from a URL."""
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
    req = requests.get(url, headers=headers)
    return req.text

def get_image(html):
    """Scrape share image."""
    image = []
    if html.find("meta", property="image"):
        image.append(html.find("meta", property="image").get('content'))
    elif html.find("meta", property="og:image"):
        image.append(html.find("meta", property="og:image").get('content'))
    elif html.find("meta", property="twitter:image"):
        image.append(html.find("meta", property="twitter:image").get('content'))
    elif html.find("img", src=True):
        for img in html.find_all("img"):
            image.append(img.get('src'))
    return image
    
def download_images_stream(links, save_path):
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
    if not os.path.exists(save_path):
        os.mkdir(save_path)
    def download_stream(link):
        try:
            r=requests.get(link, headers=headers, stream=True)
            r.raw.decode_content = True # handle spurious Content-Encoding
            img = Image.open(r.raw)
            bits = str(imagehash.average_hash(img))
            img_file = os.path.join(save_path, bits+".jpg")
            img.save(img_file)
        except:
            pass
    Pool(10).map(download_stream, links)

In [7]:
files = glob("facebook/*.xlsx")

In [23]:
web_links = []
for f in files:
    for n in range(5):
        try:
            df = pd.read_excel(f, sheet_name=n,header = 2)
            print(len(df[df["type"]=="Photo"]), f)
            for link in df[df["type"]=="Photo"]["link"]:
                web_links.append(link)
        except:
            pass
#         try:
#             df = pd.read_excel(f, sheet_name=n,header = 2)
#             print(len(df[df["Type"]=="Photo"]), f)
#             for link in df[df["Type"]=="Photo"]["Link"]:
#                 web_links.append(link)
#         except:
#             pass

159 facebook/facebook (5).xlsx
167 facebook/facebook (7).xlsx
265 facebook/facebook (1).xlsx
245 facebook/facebook (6).xlsx
31 facebook/facebook.xlsx
193 facebook/facebook (2).xlsx


In [36]:
print(len(web_links))
count = 0

298


In [45]:
from ipywidgets import IntProgress
from IPython.display import display
import time

max_count = len(web_links)
f = IntProgress(value=count, min=0, max=max_count) # instantiate the bar
display(f) # display the bar
for i, link in enumerate(web_links):
    if i >= count:
        req = requests.get("http://0.0.0.0:8889/?url={}".format(link))
        count += 1
    f.value += 1

IntProgress(value=111, max=298)